In [1]:
import sys
sys.path.insert(0, '/home/arfa/py-spinnaker2/snnTorch')
import snntorch as snn
import torch
import nir
from snntorch import export_to_nir, import_from_nir

### Generating an example NIR Graph from snntorch model:

In [2]:
# 2D sample data 
sample_data = torch.randn(1, 1, 28, 28)  #input image size is 28x28 and 1 channel #the dimension here is batch_size*number_of_channels*height*width

class NetWithAvgPool(torch.nn.Module):
    def __init__(self):
        super(NetWithAvgPool, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.lif1 = snn.Leaky(beta=0.9, init_hidden=True)
        self.fc1 = torch.nn.Linear(28*28*16 // 4, 500)  # Adjusting input size after pooling layer
        self.lif2 = snn.Leaky(beta=0.9, init_hidden=True, output=True)

    def forward(self, x):
        x = torch.nn.functional.avg_pool2d(self.conv1(x), kernel_size=2, stride=2)  #  kernel_size=2 and stride=2 for avg_pool2d
        x = x.view(-1, 28*28*16 // 4)  # Adjusting the view based on the output shape after pooling
        x = self.lif1(x)
        x = self.fc1(x)
        x = self.lif2(x)
        return x

net_with_avgpool = NetWithAvgPool()
nir_graphtest = export_to_nir(net_with_avgpool, sample_data, ignore_dims=[0])

In [ ]:
nir_graphtest.infer_types()

In [ ]:
nir.write(f"/home/arfa/py-spinnaker2/snnTorch/examples/testconv2d+avgpool.nir", nir_graphtest)

In [3]:
nir_graph = nir.read("testconv2d+avgpool.nir")

In [ ]:
#dict_keys(['conv1', 'fc1', 'input', 'lif1', 'lif2', 'output'])
#this is the node that is causing the error , because it has input_type={'input': array([] ,, and I guess also output_type={'output': array([]
nir_graph.nodes['lif1']

In [4]:
nir_graph.nodes['lif1']

LIF(tau=0.0009999997615814777, r=9.999997615814776, v_leak=0.0, v_threshold=1.0, input_type={'input': array([], dtype=float64)}, output_type={'output': array([], dtype=float64)}, metadata={})

In [ ]:
nir_graph.nodes.keys()

In [ ]:
nir_graph.edges

### Converting the NIR graph to Spinnaker2:

1. Load NIR graph

In [5]:
import nir
from spinnaker2 import hardware, s2_nir

model_path = "/home/arfa/py-spinnaker2/snnTorch/examples/testconv2d+avgpool.nir"
nir_graph = nir.read(model_path)

2. Convert NIR graph into py-spinnaker2 network

In [6]:
nir_graph.edges

[('fc1', 'lif2'),
 ('lif2', 'output'),
 ('lif1', 'fc1'),
 ('conv1', 'output'),
 ('input', 'conv1')]

In [7]:
# Configuration for converting NIR graph to SpiNNaker2
conversion_cfg = s2_nir.ConversionConfig()
conversion_cfg.output_record = ["v", "spikes"]
conversion_cfg.dt = 0.0001
conversion_cfg.conn_delay = 0
conversion_cfg.scale_weights = False # Scale weights to dynamic range on chip
#if set to True the weights will be scaled
conversion_cfg.reset = s2_nir.ResetMethod.ZERO # Reset voltage to zero at spike
conversion_cfg.integrator = s2_nir.IntegratorMethod.FORWARD # Euler-Forward

net, inp, outp = s2_nir.from_nir(nir_graph, conversion_cfg)

INFO:spinnaker2.s2_nir:from_nir(): create spinnaker2.Network from NIR graph


Node 'conv1'
Got <class 'nir.ir.conv.Conv2d'>
input:  [('input', <class 'nir.ir.graph.Input'>)]
output:  [('output', <class 'nir.ir.graph.Output'>)]

Node 'fc1'
Got <class 'nir.ir.linear.Affine'>
input:  [('lif1', <class 'nir.ir.neuron.LIF'>)]
output:  [('lif2', <class 'nir.ir.neuron.LIF'>)]

Node 'input'
Got <class 'nir.ir.graph.Input'>
this is the size of the population: 784
input:  []
output:  [('conv1', <class 'nir.ir.conv.Conv2d'>)]

Node 'lif1'
Got <class 'nir.ir.neuron.LIF'>
this is the accumulated bias: [0.]
record:  []
for node: LIF(tau=0.0009999997615814777, r=9.999997615814776, v_leak=0.0, v_threshold=1.0, input_type={'input': array([], dtype=float64)}, output_type={'output': array([], dtype=float64)}, metadata={}) this is the input_type: [250]
this is the input shape: [250]
and this is the size: 250
[250] -> []
this is the size of the population: 250
this is the created population: <spinnaker2.snn.Population object at 0x7f5391397d90>
input:  []
output:  [('fc1', <class 'nir

ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (500,)